In [16]:
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pytz
import datetime as datetime
import os
import sys
import numpy as np
import plotly.express as px


In [17]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'FPL')))


In [18]:
from FPL.fpl_api_collection import (
    get_bootstrap_data,
    get_current_gw,
    get_fixt_dfs,
    get_league_table,
    get_fixture_dfs,
    get_fixture_data
)
from FPL.fpl_utils import (
    define_sidebar,
    get_annot_size,
    map_float_to_color,
    get_text_color_from_hash,
    get_rotation
)
from FPL.fpl_params import (
    TIMEZONES_BY_CONTINENT,
    AUTHOR_CONTINENT,
    AUTHOR_CITY
)

In [19]:
team_fdr_df, team_fixt_df, team_ga_df, team_gf_df = get_fixt_dfs()

events_df = pd.DataFrame(get_bootstrap_data()['events'])

gw_min = min(events_df['id'])
gw_max = max(events_df['id'])

ct_gw = get_current_gw()

In [20]:
fixt = team_fixt_df.reset_index()
drf = team_fdr_df.reset_index()
ga = team_ga_df.reset_index()
gf = team_gf_df.reset_index()

# Rename the first column to 'Team'
fixt.rename(columns={0: 'Team'}, inplace=True)
drf.rename(columns={0: 'Team'}, inplace=True)
ga.rename(columns={0: 'Team'}, inplace=True)
gf.rename(columns={0: 'Team'}, inplace=True)

In [21]:
merged_df = pd.merge(fixt, drf, on="Team", suffixes=('_Fixture', '_FDR'))

# Convert to a format suitable for display
merged_display_df = merged_df.copy()
merged_display_df.columns = ["Team"] + [f"{gw}_Fixture" for gw in merged_display_df.columns[1:4]] + [f"{gw}_FDR" for gw in merged_display_df.columns[4:]]

# Streamlit app
st.title("FPL Fixture and Difficulty Rating Analysis")

# Display the merged DataFrame
st.markdown("### Fixture Details with Fixture Difficulty Ratings")
st.dataframe(merged_display_df)

2024-10-17 20:00:36.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:00:36.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:00:36.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:00:36.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:00:36.414 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-17 20:00:36.414 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [22]:
saaaa=get_fixture_data()

In [23]:
import json

fixtures_df = pd.DataFrame(saaaa)

fixtures_df.drop(columns='stats', inplace=True)


fixtures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   code                    380 non-null    int64  
 1   event                   380 non-null    int64  
 2   finished                380 non-null    bool   
 3   finished_provisional    380 non-null    bool   
 4   id                      380 non-null    int64  
 5   kickoff_time            380 non-null    object 
 6   minutes                 380 non-null    int64  
 7   provisional_start_time  380 non-null    bool   
 8   started                 380 non-null    bool   
 9   team_a                  380 non-null    int64  
 10  team_a_score            70 non-null     float64
 11  team_h                  380 non-null    int64  
 12  team_h_score            70 non-null     float64
 13  team_h_difficulty       380 non-null    int64  
 14  team_a_difficulty       380 non-null    in

In [24]:
teams_df = pd.DataFrame(get_bootstrap_data()['teams'])

In [25]:
teams_df = pd.DataFrame(get_bootstrap_data()['teams'])
teams_df

teams_df['logo_url'] = "https://resources.premierleague.com/premierleague/badges/70/t" + teams_df['code'].astype(str) + ".png"


team_df

'https://resources.premierleague.com/premierleague/badges/70/t'